<a href="https://colab.research.google.com/github/naholav/InternshipDataViz/blob/main/DataVisualization_Proje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download Indian intership dataset
path = kagglehub.dataset_download("khushipitroda/internshala-jobs-dataset-with-5000-rows")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/khushipitroda/internshala-jobs-dataset-with-5000-rows/versions/1


In [ ]:
import pandas as pd
import re

# Load the dataset
path = "/root/.cache/kagglehub/datasets/khushipitroda/internshala-jobs-dataset-with-5000-rows/versions/1/Jobs.csv"
data = pd.read_csv(path)

# Convert the salary column to numeric values
def process_salary(salary):
    numbers = re.findall(r"[\d\.]+", str(salary))
    if len(numbers) == 2:
        return (float(numbers[0]) + float(numbers[1])) / 2
    elif len(numbers) == 1:
        return float(numbers[0])
    else:  # If no numerical data exists
        return None

# Create a new salary column
data["salary_processed"] = data["salary"].apply(process_salary)

# Fill missing location data
data["location"].fillna("Unknown", inplace=True)

# Fill missing values in the 'Type_of_job' column with 'Unknown'
data['Type_of_job'] = data['Type_of_job'].fillna('Unknown')

# Drop the 'actively_hiring' column
data.drop(columns=["actively_hiring"], inplace=True)

# Clean missing processed salary data
data = data.dropna(subset=['salary_processed'])

# View the first few rows of the processed dataset
print(data.head())

# Save processed dataset
data.to_csv("/root/processed_dataset.csv", index=False)
print("Dataset successfully processed and saved!")


   Unnamed: 0                            Type_of_job  \
0           0         Business Development Executive   
1           1                       Graphic Designer   
2           2              Corporate Sales Associate   
3           3  Assistant Manager - Sales (Delhi NCR)   
4           4         Business Development Executive   

                                   company_name   location          salary  \
0  Ifisys Software And Services Private Limited      Delhi  ₹  2 - 2.5 LPA   
1                                  Backbenchers    Gurgaon  ₹  2 - 3.5 LPA   
2                       FYIPEN Private Limitted      Delhi  ₹  2.5 - 4 LPA   
3                               Marketing Works  Faridabad  ₹  4 - 5.5 LPA   
4           Bizcon Fintech Labs Private Limited      Noida  ₹  2 - 3.2 LPA   

  experience  salary_processed  
0  0-2 years              2.25  
1  0-2 years              2.75  
2  0-2 years              3.25  
3  0-5 years              4.75  
4  0-2 years              2.6

<ipython-input-34-1ff318df72f8>:22: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [ ]:
import kagglehub

# Download the Indian location dataset
path = kagglehub.dataset_download("parulpandey/indian-cities-database")

print("Path to dataset files:", path)

import pandas as pd

# Load the data
df = pd.read_csv('/root/.cache/kagglehub/datasets/parulpandey/indian-cities-database/versions/4/Indian Cities Database.csv')

# Check the head of data
df.head()





Path to dataset files: /root/.cache/kagglehub/datasets/parulpandey/indian-cities-database/versions/4


,City,Lat,Long,country,iso2,State
0,Abohar,30.144533,74.195520,India,IN,Punjab
1,Adilabad,19.400000,78.310000,India,IN,Telangana
2,Agartala,23.836049,91.279386,India,IN,Tripura
3,Agra,27.187935,78.003944,India,IN,Uttar Pradesh
4,Ahmadnagar,19.094571,74.738432,India,IN,Maharashtra


In [ ]:
import folium

# Start the map centered on India
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

# Add a marker on the map for each city
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],  # Latitude and longitude
        popup=row['City'],  # Popup content
    ).add_to(india_map)

# Save the raw map as an HTML file
india_map.save("/root/raw_indian_cities_map.html")

from google.colab import files

# Download HTML file
files.download('/root/raw_indian_cities_map.html')




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import folium
import branca

# Calculate the number of internships per city
city_counts = data['location'].value_counts().reset_index()
city_counts.columns = ['City', 'Internship_Count']

# Filter out cities with less than 5 internships
city_counts = city_counts[city_counts['Internship_Count'] >= 5]

# Centering the map on India
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

# Define custom intervals and corresponding colors
intervals = [5, 30, 100, 274, 411, 571, 684]
colors = ['red', 'orange', 'yellow', 'purple', 'blue', 'brown']

# Creating a StepColormap
color_scale = branca.colormap.StepColormap(
    colors=colors,
    index=intervals,
    vmin=min(intervals),
    vmax=max(intervals)
)
color_scale.caption = 'Internship Count (Red: 5-30, Brown: 684)\nCities with less than 5 internships are not shown'

# Adding cities to the map
for index, row in city_counts.iterrows():
    city = row['City']
    internship_count = row['Internship_Count']

    # Matching the city name with the rows in the dataset
    city_data = df.loc[df['City'] == city]

    if not city_data.empty:  # If city data is found
        lat = city_data['Lat'].values[0]
        long = city_data['Long'].values[0]

        # Keep the circle size fixed
        folium.CircleMarker(
            location=[lat, long],
            radius=8,  # Fixed circle size for each city
            popup=f"{city}: {internship_count} Internship Posts",
            color='black',  # Black border
            weight=1,
            fill=True,
            fill_color=color_scale(internship_count),
            fill_opacity=0.9
        ).add_to(india_map)

# Adding the color scale to the map
india_map.add_child(color_scale)

# Saving the map
india_map.save('/root/visualization_indian_cities_map.html')

# Allowing the user to download the visualization
from google.colab import files
files.download('/root/visualization_indian_cities_map.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import re
import pandas as pd
import plotly.express as px

# Sector categorization function
def categorize_sector(sector):
    sector = str(sector).lower()  # Standardize data by converting to lowercase

    # Keywords related to IT/Tech
    if re.search(r"\b(develop|developer|development|software|programmer|engineer|it|tech|coding|data|data scientist|web|app|cloud|network|machine learning|ai|artificial intelligence|database|python|java|ruby|javascript|sql|frontend|backend|system administrator)\b", sector):
        return "IT/Tech"

    # Keywords related to Creative
    elif re.search(r"\b(design|designer|artist|editor|creative|animation|graphics|illustrator|photography|videography|motion|visual effects|ux|ui|branding|fashion|interior design)\b", sector):
        return "Creative"

    # Keywords related to Marketing/Sales
    elif re.search(r"\b(marketing|sales|business|management|advertising|seo|brand|promotion|digital marketing|content|social media|lead generation|public relations|business development|product manager)\b", sector):
        return "Marketing/Sales"

    # Keywords related to Education/Teaching
    elif re.search(r"\b(teach|teacher|education|trainer|tutor|mentor|academy|school|instructor|educator|curriculum|online learning|elearning|classroom|training|coaching|professor|lecturer)\b", sector):
        return "Education/Teaching"

    # Keywords related to Healthcare
    elif re.search(r"\b(health|medicine|medical|nurse|pharma|therapy|clinic|doctor|dentist|hospital|pharmacist|healthcare|nutritionist|physiotherapist|psychologist|therapist|radiologist|biomedical|biotech)\b", sector):
        return "Healthcare"

    # Keywords related to Finance
    elif re.search(r"\b(finance|accounting|bank|investment|economics|audit|tax|insurance|financial analyst|stock|portfolio|capital|banking|credit|financial advisor|corporate finance|real estate|wealth management)\b", sector):
        return "Finance"

    # Keywords related to Legal
    elif re.search(r"\b(lawyer|attorney|legal|paralegal|litigation|contract|court|law|legal advisor|barrister|solicitor|advocate|judge)\b", sector):
        return "Legal"

    # Keywords related to Engineering
    elif re.search(r"\b(engineering|engineer|mechanical|electrical|civil|chemical|structural|aerospace|industrial|electronic|construction|project manager|systems engineer|design engineer)\b", sector):
        return "Engineering"

    # Keywords related to Hospitality/Tourism
    elif re.search(r"\b(hospitality|tourism|hotel|restaurant|catering|travel|chef|tour guide|event planning|guest services|resort|customer service)\b", sector):
        return "Hospitality/Tourism"

    # Keywords related to Retail
    elif re.search(r"\b(retail|store|sales associate|customer service|merchandising|shopping|supermarket|fashion retail|e-commerce|sales)\b", sector):
        return "Retail"

    # Keywords related to Manufacturing
    elif re.search(r"\b(manufacturing|production|factory|plant|assembly|operation|process engineer|quality control|supply chain|logistics)\b", sector):
        return "Manufacturing"

    else:
        return "Other"

# Create the 'sector_category' column
data['sector_category'] = data['Type_of_job'].apply(categorize_sector)

# Calculate salary and job counts for each sector
sector_stats = data.groupby('sector_category').agg(
    average_salary=('salary_processed', 'mean'),
    internship_count=('sector_category', 'size')
).reset_index()

# Round average salary to 2 decimal places
sector_stats['average_salary'] = sector_stats['average_salary'].round(2)

# Select the top 10 sectors
top_10_sectors = sector_stats.nlargest(10, 'internship_count')

# Create Bubble Chart
fig = px.scatter(
    top_10_sectors,
    x='sector_category',
    y='average_salary',
    size='internship_count',
    color='average_salary',
    title="Top 10 Sectors: Average Salary and Job Count",
    labels={
        'sector_category': 'Sector Category',
        'average_salary': 'Average Salary (LPA)',
        'internship_count': 'Internship Count'
    },
    hover_data={'average_salary': ':.2f', 'internship_count': True}
)

# Customize visualization by increasing bubble size
fig.update_traces(
    marker=dict(
        opacity=0.8,
        line=dict(width=1, color='DarkSlateGrey'),
        size=top_10_sectors['internship_count'] * 20 # This multiplies the size by 20
    )
)
fig.update_layout(
    title_font_size=18
)

# Increase the size of the Bubble chart
fig.update_layout(
    height=800,
    width=1500,
)

# Add annotation to the chart
fig.add_annotation(
    x=1.1, y=1.1,
    text="1 LPA = 100,000 INR.",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="center",
    xref="paper", yref="paper"
)

# Show the visualization
fig.show()


In [ ]:
import re
import pandas as pd
import plotly.express as px

# Categorize "work from home" and "in-person" values in the 'location' column
def categorize_work_type(location):
    location = str(location).lower()
    if re.search(r'work\s*from\s*home', location):
        return "Remote"
    else:
        return "In-person"

# Categorize work type
data['work_type'] = data['location'].apply(categorize_work_type)

# Sector categorization function
def categorize_sector(sector):
    sector = str(sector).lower()  # Standardize data by converting to lowercase

    # Keywords related to IT/Tech
    if re.search(r"\b(develop|developer|development|software|programmer|engineer|it|tech|coding|data|data scientist|web|app|cloud|network|machine learning|ai|artificial intelligence|database|python|java|ruby|javascript|sql|frontend|backend|system administrator)\b", sector):
        return "IT/Tech"

    # Keywords related to Creative
    elif re.search(r"\b(design|designer|artist|editor|creative|animation|graphics|illustrator|photography|videography|motion|visual effects|ux|ui|branding|fashion|interior design)\b", sector):
        return "Creative"

    # Keywords related to Marketing/Sales
    elif re.search(r"\b(marketing|sales|business|management|advertising|seo|brand|promotion|digital marketing|content|social media|lead generation|public relations|business development|product manager)\b", sector):
        return "Marketing/Sales"

    # Keywords related to Education/Teaching
    elif re.search(r"\b(teach|teacher|education|trainer|tutor|mentor|academy|school|instructor|educator|curriculum|online learning|elearning|classroom|training|coaching|professor|lecturer)\b", sector):
        return "Education/Teaching"

    # Keywords related to Healthcare
    elif re.search(r"\b(health|medicine|medical|nurse|pharma|therapy|clinic|doctor|dentist|hospital|pharmacist|healthcare|nutritionist|physiotherapist|psychologist|therapist|radiologist|biomedical|biotech)\b", sector):
        return "Healthcare"

    # Keywords related to Finance
    elif re.search(r"\b(finance|accounting|bank|investment|economics|audit|tax|insurance|financial analyst|stock|portfolio|capital|banking|credit|financial advisor|corporate finance|real estate|wealth management)\b", sector):
        return "Finance"

    # Keywords related to Legal
    elif re.search(r"\b(lawyer|attorney|legal|paralegal|litigation|contract|court|law|legal advisor|barrister|solicitor|advocate|judge)\b", sector):
        return "Legal"

    # Keywords related to Engineering
    elif re.search(r"\b(engineering|engineer|mechanical|electrical|civil|chemical|structural|aerospace|industrial|electronic|construction|project manager|systems engineer|design engineer)\b", sector):
        return "Engineering"

    # Keywords related to Hospitality/Tourism
    elif re.search(r"\b(hospitality|tourism|hotel|restaurant|catering|travel|chef|tour guide|event planning|guest services|resort|customer service)\b", sector):
        return "Hospitality/Tourism"

    # Keywords related to Retail
    elif re.search(r"\b(retail|store|sales associate|customer service|merchandising|shopping|supermarket|fashion retail|e-commerce|sales)\b", sector):
        return "Retail"

    # Keywords related to Manufacturing
    elif re.search(r"\b(manufacturing|production|factory|plant|assembly|operation|process engineer|quality control|supply chain|logistics)\b", sector):
        return "Manufacturing"

    else:
        return "Other"

# Apply categorization for work type and sector
data['sector_category'] = data['Type_of_job'].apply(categorize_sector)

# Count the number of internships per work type and sector category
sector_counts = data.groupby(['work_type', 'sector_category']).size().reset_index(name='internship_count')

# Filter out the sector counts with internship_count less than 25
sector_counts = sector_counts[sector_counts['internship_count'] >= 25]

# Calculate the average salary for each work type and sector category
avg_salaries = data.groupby(['work_type', 'sector_category'])['salary_processed'].mean().reset_index(name='avg_salary')

# Merge both dataframes
sunburst_data = pd.merge(sector_counts, avg_salaries, on=['work_type', 'sector_category'])

# Create the Sunburst chart
fig = px.sunburst(
    sunburst_data,
    path=['work_type', 'sector_category'],  # Parent: work_type, Child: sector_category
    values='internship_count',  # Size of the bubbles
    title="Internship Distribution by Work Type and Sector",
    hover_data={'avg_salary': True},  # Show avg_salary in the hover popup
    labels={'work_type': 'Work Type', 'sector_category': 'Sector Category', 'internship_count': 'Internship Count', 'avg_salary': 'Average Salary (LPA)'}
)

# Add hover text to show the average salary
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>Internship Count: %{value}<br>Average Salary: %{customdata[0]:,.2f} LPA'
)

# Add annotation to the chart
fig.add_annotation(
    x=0.5, y=-0.1,
    text="1 LPA = 100,000 INR.",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="center",
    xref="paper", yref="paper"
)

# Increase the size of the Sunburst chart
fig.update_layout(
    height=800,  # Increase height
    width=1500,  # Increase width
)

# Add annotation to show that sectors with less than 25 internships are excluded
fig.add_annotation(
    x=1.03, y=1.10,
    text="Sectors with internship count < 25 are excluded",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="center",
    xref="paper", yref="paper"
)

# Show the chart
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import re

# Sector categorization function
def categorize_sector(sector):
    sector = str(sector).lower()  # Standardize data by converting to lowercase

    # Keywords related to IT/Tech
    if re.search(r"\b(develop|developer|development|software|programmer|engineer|it|tech|coding|data|data scientist|web|app|cloud|network|machine learning|ai|artificial intelligence|database|python|java|ruby|javascript|sql|frontend|backend|system administrator)\b", sector):
        return "IT/Tech"

    # Keywords related to Creative
    elif re.search(r"\b(design|designer|artist|editor|creative|animation|graphics|illustrator|photography|videography|motion|visual effects|ux|ui|branding|fashion|interior design)\b", sector):
        return "Creative"

    # Keywords related to Marketing/Sales
    elif re.search(r"\b(marketing|sales|business|management|advertising|seo|brand|promotion|digital marketing|content|social media|lead generation|public relations|business development|product manager)\b", sector):
        return "Marketing/Sales"

    # Keywords related to Education/Teaching
    elif re.search(r"\b(teach|teacher|education|trainer|tutor|mentor|academy|school|instructor|educator|curriculum|online learning|elearning|classroom|training|coaching|professor|lecturer)\b", sector):
        return "Education/Teaching"

    # Keywords related to Healthcare
    elif re.search(r"\b(health|medicine|medical|nurse|pharma|therapy|clinic|doctor|dentist|hospital|pharmacist|healthcare|nutritionist|physiotherapist|psychologist|therapist|radiologist|biomedical|biotech)\b", sector):
        return "Healthcare"

    # Keywords related to Finance
    elif re.search(r"\b(finance|accounting|bank|investment|economics|audit|tax|insurance|financial analyst|stock|portfolio|capital|banking|credit|financial advisor|corporate finance|real estate|wealth management)\b", sector):
        return "Finance"

    # Keywords related to Legal
    elif re.search(r"\b(lawyer|attorney|legal|paralegal|litigation|contract|court|law|legal advisor|barrister|solicitor|advocate|judge)\b", sector):
        return "Legal"

    # Keywords related to Engineering
    elif re.search(r"\b(engineering|engineer|mechanical|electrical|civil|chemical|structural|aerospace|industrial|electronic|construction|project manager|systems engineer|design engineer)\b", sector):
        return "Engineering"

    # Keywords related to Hospitality/Tourism
    elif re.search(r"\b(hospitality|tourism|hotel|restaurant|catering|travel|chef|tour guide|event planning|guest services|resort|customer service)\b", sector):
        return "Hospitality/Tourism"

    # Keywords related to Retail
    elif re.search(r"\b(retail|store|sales associate|customer service|merchandising|shopping|supermarket|fashion retail|e-commerce|sales)\b", sector):
        return "Retail"

    # Keywords related to Manufacturing
    elif re.search(r"\b(manufacturing|production|factory|plant|assembly|operation|process engineer|quality control|supply chain|logistics)\b", sector):
        return "Manufacturing"

    else:
        return "Other"

# Apply sector categorization to your data
data['sector_category'] = data['Type_of_job'].apply(categorize_sector)

# Group by company and sector to count internships and calculate average salary
company_sector_counts = data.groupby(['company_name', 'sector_category']).agg(
    internship_count=('company_name', 'size'),
    average_salary=('salary_processed', 'mean')
).reset_index()

# Get the top 10 companies with the most internships
top_companies = company_sector_counts.groupby('company_name').sum().nlargest(10, 'internship_count').reset_index()

# Merge back the sector counts for these companies
top_companies_sector = company_sector_counts[company_sector_counts['company_name'].isin(top_companies['company_name'])]

# Treemap visualization with improved layout
fig = px.treemap(
    top_companies_sector,
    path=['company_name', 'sector_category'],  # Hierarchy: Company -> Sector
    values='internship_count',  # Size of rectangles: Number of internships
    color='average_salary',  # Use average salary for coloring
    color_continuous_scale='Sunset',  # Changed to a more vibrant color scale
    title='Top 10 Companies and Their Internship Sectors with Average Salary (1 LPA = 100,000 INR.)',
    hover_data={'internship_count': True, 'average_salary': True},  # Display internship count and average salary on hover
    labels={'company_name': 'Company Name', 'sector_category': 'Sector', 'internship_count': 'Number of Internships', 'average_salary': 'Average Salary (LPA)'}
)

# Customize the layout for a cleaner look
fig.update_layout(
    margin=dict(t=50, b=50, l=50, r=50),  # Reduce the margins for a tighter layout
    plot_bgcolor='white',  # Set background color to white
    title_font=dict(size=18, family='Arial, sans-serif', color='black'),  # Title font customization
    font=dict(size=12, color='black'),  # General font customization
    paper_bgcolor='white',  # Set paper background color to white
)

# Update trace to remove borders around the rectangles and make them more visible
fig.update_traces(
    marker=dict(line=dict(width=1.3, color='black'))
)

# Increase the size of the Treemap chart
fig.update_layout(
    height=800,
    width=1500,
)

# Show the plot
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import re

# Sector categorization function
def categorize_sector(sector):
    sector = str(sector).lower()  # Standardize data by converting to lowercase

    # Keywords related to IT/Tech
    if re.search(r"\b(develop|developer|development|software|programmer|engineer|it|tech|coding|data|data scientist|web|app|cloud|network|machine learning|ai|artificial intelligence|database|python|java|ruby|javascript|sql|frontend|backend|system administrator)\b", sector):
        return "IT/Tech"

    # Keywords related to Creative
    elif re.search(r"\b(design|designer|artist|editor|creative|animation|graphics|illustrator|photography|videography|motion|visual effects|ux|ui|branding|fashion|interior design)\b", sector):
        return "Creative"

    # Keywords related to Marketing/Sales
    elif re.search(r"\b(marketing|sales|business|management|advertising|seo|brand|promotion|digital marketing|content|social media|lead generation|public relations|business development|product manager)\b", sector):
        return "Marketing/Sales"

    # Keywords related to Education/Teaching
    elif re.search(r"\b(teach|teacher|education|trainer|tutor|mentor|academy|school|instructor|educator|curriculum|online learning|elearning|classroom|training|coaching|professor|lecturer)\b", sector):
        return "Education/Teaching"

    # Keywords related to Healthcare
    elif re.search(r"\b(health|medicine|medical|nurse|pharma|therapy|clinic|doctor|dentist|hospital|pharmacist|healthcare|nutritionist|physiotherapist|psychologist|therapist|radiologist|biomedical|biotech)\b", sector):
        return "Healthcare"

    # Keywords related to Finance
    elif re.search(r"\b(finance|accounting|bank|investment|economics|audit|tax|insurance|financial analyst|stock|portfolio|capital|banking|credit|financial advisor|corporate finance|real estate|wealth management)\b", sector):
        return "Finance"

    # Keywords related to Legal
    elif re.search(r"\b(lawyer|attorney|legal|paralegal|litigation|contract|court|law|legal advisor|barrister|solicitor|advocate|judge)\b", sector):
        return "Legal"

    # Keywords related to Engineering
    elif re.search(r"\b(engineering|engineer|mechanical|electrical|civil|chemical|structural|aerospace|industrial|electronic|construction|project manager|systems engineer|design engineer)\b", sector):
        return "Engineering"

    # Keywords related to Hospitality/Tourism
    elif re.search(r"\b(hospitality|tourism|hotel|restaurant|catering|travel|chef|tour guide|event planning|guest services|resort|customer service)\b", sector):
        return "Hospitality/Tourism"

    # Keywords related to Retail
    elif re.search(r"\b(retail|store|sales associate|customer service|merchandising|shopping|supermarket|fashion retail|e-commerce|sales)\b", sector):
        return "Retail"

    # Keywords related to Manufacturing
    elif re.search(r"\b(manufacturing|production|factory|plant|assembly|operation|process engineer|quality control|supply chain|logistics)\b", sector):
        return "Manufacturing"

    else:
        return "Other"

# Apply sector categorization to your data
data['sector_category'] = data['Type_of_job'].apply(categorize_sector)

# Filter out rows where location is "Work from Home" or similar
data = data[~data['location'].str.contains('remote|virtual|work from home', case=False, na=False)]

# Group by city, sector to count internships and calculate average salary
city_sector_counts = data.groupby(['location', 'sector_category']).agg(
    internship_count=('company_name', 'size'),
    average_salary=('salary_processed', 'mean')
).reset_index()

# Get the top 10 cities with the most internships
top_cities = city_sector_counts.groupby('location').sum().nlargest(10, 'internship_count').reset_index()

# Merge back the sector counts for these cities
top_cities_sector = city_sector_counts[city_sector_counts['location'].isin(top_cities['location'])]

# Get the top 5 sectors for the selected cities
top_sectors = top_cities_sector.groupby('sector_category').sum().nlargest(5, 'internship_count').reset_index()

# Merge back the sector counts for these top 5 sectors
final_data = top_cities_sector[top_cities_sector['sector_category'].isin(top_sectors['sector_category'])]

# Create a bar plot
fig = px.bar(
    final_data,
    x='location',  # X axis: City
    y='internship_count',  # Y axis: Internship Count
    color='sector_category',  # Color based on Sector
    hover_data=['average_salary'],  # Show average salary on hover
    title="Top 10 Cities with Most Internships by Top 5 Sectors",
    labels={'location': 'City', 'internship_count': 'Number of Internships'},
    category_orders={'sector_category': top_sectors['sector_category'].tolist()},
)

# Customize the layout
fig.update_layout(
    title="Top 10 Cities with Most Internships by Top 5 Sectors",
    xaxis_title="City",
    yaxis_title="Number of Internships",
    barmode="stack",  # Stack the bars
    height=600,
    width=1200,
)

# Add the note at the bottom
fig.add_annotation(
    text="1 LPA = 100,000 INR",
    xref="paper", yref="paper",
    x=0.5, y=-0.1,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

# Increase the size of the Stacked Bar Chart
fig.update_layout(
    height=800,
    width=1500,
)

# Show the plot
fig.show()

